In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepc as qem

In [2]:
# we introduced noise into the network and trained it without any error mitigation techniques
# to observe the behavior of the Loss function degradation during training.

n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)

nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
pauli_set_of_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_of_n_qubits)

nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.9)
nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)

@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_without_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

def cost(x):
    return ((train_cir_without_qem(x))-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 100
params = weights

for i in range(steps):
    nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.85)
    nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)
    params = opt.step(cost, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.3277493
Cost after step    10:  0.1620262
Cost after step    15:  0.0834116
Cost after step    20:  0.0310678
Cost after step    25:  0.0550540
Cost after step    30:  0.0473296
Cost after step    35:  0.0637228
Cost after step    40:  0.0305615
Cost after step    45:  0.0528701
Cost after step    50:  0.0499776
Cost after step    55:  0.0686858
Cost after step    60:  0.0540521
Cost after step    65:  0.0395443
Cost after step    70:  0.0564822
Cost after step    75:  0.0124367
Cost after step    80:  0.0513430
Cost after step    85:  0.0147628
Cost after step    90:  0.0386811
Cost after step    95:  0.0324363
Cost after step   100:  0.0363526
Optimized rotation angles: [[[ 0.816732    1.54391133  0.01661329]
  [ 0.64230455  1.93650413  0.24587008]]

 [[-0.24514998  1.20448349  0.77630559]
  [ 0.20841162  0.44672854  0.92420115]]]


In [3]:
# When simulating a complex scenario of time-varying noise channels,
# it can be observed that the descent of the Loss function is relatively unstable
# in the absence of error mitigation techniques.

In [4]:
# We estimate the total noise 'p_t' using the QEMZMSEPC method,
# allowing us to incorporate noise 'p_t' into the definition of the Loss function,
# thereby mitigating the impact of noise on quantum machine learning tasks.

operations = ['RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT',
              'RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT']
shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)
w = weights.copy().reshape(-1)
paras = [[0, w[0]], [0, w[1]], [0, w[2]], [0, w[3]], [0, w[4]], [0, w[5]], [0, 1], [1, 0],
     [0, w[6]], [0, w[7]], [0, w[8]], [0, w[9]], [0, w[10]], [0, w[11]], [0, 1], [1, 0]]
qemzmsepc = qem.QEMZMSEPC(n_qubits)
_, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                         kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
print(p_t)

0.8096893796064297


In [5]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_with_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

# We modify the definition of the Loss function
# by dividing the output of the quantum circuit by the noise parameter p_t
# to obtain the expected output of the quantum circuit after error mitigation,
# and then proceed with the Loss calculation.

def cost(x):
    return ((train_cir_with_qem(x) / p_t) - (-1)) ** 2

opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 100
params = weights

for i in range(steps):
    params = opt.step(cost, params)
    w = params.copy().reshape(-1)
    paras = [[0, w[0]], [0, w[1]], [0, w[2]], [0, w[3]], [0, w[4]], [0, w[5]], [0, 1], [1, 0],
         [0, w[6]], [0, w[7]], [0, w[8]], [0, w[9]], [0, w[10]], [0, w[11]], [0, 1], [1, 0]]
    qemzmsepc = qem.QEMZMSEPC(n_qubits)
    nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.85)
    _, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                             kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.2816102
Cost after step    10:  0.0755249
Cost after step    15:  0.0363118
Cost after step    20:  0.0221546
Cost after step    25:  0.0153002
Cost after step    30:  0.0113866
Cost after step    35:  0.0089049
Cost after step    40:  0.0072133
Cost after step    45:  0.0059979
Cost after step    50:  0.0050890
Cost after step    55:  0.0043878
Cost after step    60:  0.0038329
Cost after step    65:  0.0033848
Cost after step    70:  0.0030165
Cost after step    75:  0.0027095
Cost after step    80:  0.0024503
Cost after step    85:  0.0022291
Cost after step    90:  0.0020385
Cost after step    95:  0.0018729
Cost after step   100:  0.0017279
Optimized rotation angles: [[[ 0.84813186  1.15252958  0.13755776]
  [ 0.65017847  0.69879178 -0.03966164]]

 [[ 0.10918059  2.41888637  0.92832058]
  [ 0.99151456  0.93468784  0.59093966]]]


In [6]:
# From the results, we can observe that the Loss function exhibits a stable decreasing trend
# after error mitigation through the QEMZMSEPC scheme,
# and ultimately, the outcome is better than without error mitigation.